# AMP-SOA portfolio optimization using adaptive meta-policy based on second-order agents with deep reinforcment learning
---

## 7.0 Deep Reinforcement Learning Portfolios (First order agents) and second order agent

### 7.1 Important var

In [1]:
#second-order-agent-days-neuralnetwork
days=10
neural_network = 'dense_bn'

In [ ]:
import pandas as pd
import pandas
import numpy as np
import numpy
import matplotlib
import math
import models
import os
matplotlib.use('Agg')
import datetime
from pypfopt import expected_returns, risk_models, EfficientFrontier, objective_functions
from config import config
from backtest import backtest_strat, baseline_strat
import env_portfolio
from env_portfolio import StockPortfolioEnv
from models import DRLAgent
from finrl.preprocessing.data import data_split
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.layers import Dense, LSTM, LeakyReLU, Dropout, Conv2D, MaxPooling2D, Flatten, Conv1D, MaxPooling1D
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler,StandardScaler
from keras.models import Sequential, load_model
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import seaborn as sns
from keras.optimizers import SGD
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping
import pyfolio as pf
from pyfolio import timeseries
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

best_return = True
while best_return: 
    tech_indicator_list = ['f01','f02','f03','f04']
    %store -r train_df
    %store -r test_df
    train_df.head()
    stock_dimension = len(train_df.tic.unique())
    state_space = stock_dimension
    print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")
    weights_initial = [1/stock_dimension]*stock_dimension
    env_kwargs = {
        "hmax": 500, 
        "initial_amount": 1000000, 
        "transaction_cost_pct": 0.001, 
        "state_space": state_space, 
        "stock_dim": stock_dimension, 
        "tech_indicator_list": tech_indicator_list, 
        "action_space": stock_dimension, 
        "reward_scaling": 0,
        'initial_weights': [1/stock_dimension]*stock_dimension
    }
    e_train_gym = StockPortfolioEnv(df = train_df, **env_kwargs)
    env_train, _ = e_train_gym.get_sb_env()
    print(type(env_train))
    
    # 1) DEFITION AGENTS
    ####A2C AGENT
    agent = DRLAgent(env = env_train)
    A2C_PARAMS = {"n_steps": 5, "ent_coef": 0.005, "learning_rate": 0.001}
    model_a2c = agent.get_model(model_name="a2c",model_kwargs = A2C_PARAMS)
    trained_a2c = agent.train_model(model=model_a2c,    tb_log_name='a2c',   total_timesteps=10000)
    agent = DRLAgent(env = env_train)
    PPO_PARAMS = {
        "n_steps": 2048,
        "ent_coef": 0.005,
        "learning_rate": 0.001,
        "batch_size": 100,
    }
    model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)
    trained_ppo = agent.train_model(model=model_ppo,   tb_log_name='ppo',  total_timesteps=10000)
    ####DDPG AGENT
    agent = DRLAgent(env = env_train)
    DDPG_PARAMS = {"batch_size": 100, "buffer_size": 50000, "learning_rate": 0.001,"learning_starts": 100}
    model_ddpg = agent.get_model("ddpg",model_kwargs = DDPG_PARAMS)
    trained_ddpg = agent.train_model(model=model_ddpg,  tb_log_name='ddpg', total_timesteps=10000)
    ####TD3 AGENT
    agent = DRLAgent(env=env_train)
    TD3_PARAMS = {"batch_size": 100,
                "buffer_size": 50000,
                "learning_rate": 0.001,
                "learning_starts": 100}
    model_td3 = agent.get_model("td3", model_kwargs=TD3_PARAMS)
    trained_td3 = agent.train_model(
        model=model_td3, tb_log_name='td3', total_timesteps=10000)
    ####SAC AGENT
    agent = DRLAgent(env=env_train)
    SAC_PARAMS = {
        "batch_size": 100,
        "buffer_size": 50000,
        "learning_rate": 0.001,
        "learning_starts": 100,
        "ent_coef": "auto_0.1",
    }
    model_sac = agent.get_model("sac", model_kwargs=SAC_PARAMS)
    trained_sac = agent.train_model(
        model=model_sac, tb_log_name='sac', total_timesteps=10000)
    
    # 2)Train the agents
    # A2C Train Model
    e_trade_gym = StockPortfolioEnv(df = train_df, **env_kwargs)
    env_trade, obs_trade = e_trade_gym.get_sb_env()
    a2c_train_daily_return, a2c_train_weights = DRLAgent.DRL_prediction(model=trained_a2c, test_data = train_df, test_env = env_trade, test_obs = obs_trade)
    # PPO Train Model
    e_trade_gym = StockPortfolioEnv(df = train_df, **env_kwargs)
    env_trade, obs_trade = e_trade_gym.get_sb_env()
    ppo_train_daily_return, ppo_train_weights = DRLAgent.DRL_prediction(model=trained_ppo, test_data = train_df,test_env = env_trade,test_obs = obs_trade)
    # DDPG Train Model
    e_trade_gym = StockPortfolioEnv(df = train_df, **env_kwargs)
    env_trade, obs_trade = e_trade_gym.get_sb_env()
    ddpg_train_daily_return, ddpg_train_weights = DRLAgent.DRL_prediction(model=trained_ddpg, test_data = train_df,  test_env = env_trade, test_obs = obs_trade)
    # TD3 Train Model
    e_trade_gym = StockPortfolioEnv(df=train_df, **env_kwargs)
    env_trade, obs_trade = e_trade_gym.get_sb_env()
    td3_train_daily_return, td3_train_weights = DRLAgent.DRL_prediction(
        model=trained_td3, test_data=train_df,  test_env=env_trade, test_obs=obs_trade)
    # SAC Train Model
    e_trade_gym = StockPortfolioEnv(df=train_df, **env_kwargs)
    env_trade, obs_trade = e_trade_gym.get_sb_env()
    sac_train_daily_return, sac_train_weights = DRLAgent.DRL_prediction(
        model=trained_sac, test_data=train_df,  test_env=env_trade, test_obs=obs_trade)
    # Store the Training Models
    %store a2c_train_daily_return
    %store ppo_train_daily_return
    %store ddpg_train_daily_return
    %store td3_train_daily_return
    %store sac_train_daily_return
    a2c_train_daily_return.to_csv('a2c_train_daily_return.csv',index=False)
    ppo_train_daily_return.to_csv('ppo_train_daily_return.csv',index=False)
    ddpg_train_daily_return.to_csv('ddpg_train_daily_return.csv',index=False)
    td3_train_daily_return.to_csv('td3_train_daily_return.csv',index=False)
    sac_train_daily_return.to_csv('sac_train_daily_return.csv',index=False)

    # 3) Trading (test Agents)
    # A2C Test Model
    e_trade_gym = StockPortfolioEnv(df = test_df, **env_kwargs)
    env_trade, obs_trade = e_trade_gym.get_sb_env()
    a2c_test_daily_return, a2c_test_weights = DRLAgent.DRL_prediction(model=trained_a2c, test_data = test_df, test_env = env_trade, test_obs = obs_trade)
    a2c_test_weights.to_csv('a2c_test_weights.csv')
    a2c_test_daily_return.to_csv('a2c_test_daily_return.csv')
    a2c_test_weights
    # PPO Test Model
    e_trade_gym = StockPortfolioEnv(df = test_df, **env_kwargs)
    env_trade, obs_trade = e_trade_gym.get_sb_env()
    ppo_test_daily_return, ppo_test_weights = DRLAgent.DRL_prediction(model=trained_ppo, test_data = test_df, test_env = env_trade, test_obs = obs_trade)
    ppo_test_weights.to_csv('ppo_test_weights.csv')
    ppo_test_daily_return.to_csv('ppo_test_daily_return.csv')
    # DDPG Test Model
    e_trade_gym = StockPortfolioEnv(df = test_df, **env_kwargs)
    env_trade, obs_trade = e_trade_gym.get_sb_env()
    ddpg_test_daily_return, ddpg_test_weights = DRLAgent.DRL_prediction(model=trained_ddpg, test_data = test_df, test_env = env_trade,test_obs = obs_trade)
    ddpg_test_weights.to_csv('ddpg_test_weights.csv')
    ddpg_test_daily_return.to_csv('ddpg_test_daily_return.csv')
    # TD3 Test Model
    e_trade_gym = StockPortfolioEnv(df=test_df, **env_kwargs)
    env_trade, obs_trade = e_trade_gym.get_sb_env()
    td3_test_daily_return, td3_test_weights = DRLAgent.DRL_prediction(
        model=trained_td3, test_data=test_df, test_env=env_trade, test_obs=obs_trade)
    td3_test_weights.to_csv('td3_test_weights.csv')
    td3_test_daily_return.to_csv('td3_test_daily_return.csv')
    # SAC Test Model
    e_trade_gym = StockPortfolioEnv(df=test_df, **env_kwargs)
    env_trade, obs_trade = e_trade_gym.get_sb_env()
    sac_test_daily_return, sac_test_weights = DRLAgent.DRL_prediction(
        model=trained_sac, test_data=test_df, test_env=env_trade, test_obs=obs_trade)
    sac_test_weights.to_csv('sac_test_weights.csv')
    sac_test_daily_return.to_csv('sac_test_daily_return.csv')
    ### Save the Portfolios
    a2c_test_portfolio = a2c_test_weights.copy()
    a2c_test_returns = a2c_test_daily_return.copy()
    ppo_test_portfolio = ppo_test_weights.copy()
    ppo_test_returns = ppo_test_daily_return.copy()
    ddpg_test_portfolio = ddpg_test_weights.copy()
    ddpg_test_returns = ddpg_test_daily_return.copy()
    td3_test_portfolio = td3_test_weights.copy()
    td3_test_returns = td3_test_daily_return.copy()
    sac_test_portfolio = sac_test_weights.copy()
    sac_test_returns = sac_test_daily_return.copy()
    %store a2c_test_portfolio
    %store a2c_test_returns 
    %store ppo_test_portfolio
    %store ppo_test_returns 
    %store ddpg_test_portfolio
    %store ddpg_test_returns
    %store td3_test_portfolio
    %store td3_test_returns
    %store sac_test_portfolio
    %store sac_test_returns
    
    # 4) Second order agent init
    agents_count = 5
    a2c_test_weights = pd.read_csv('a2c_test_weights.csv')
    ppo_test_weights = pd.read_csv('ppo_test_weights.csv')
    ddpg_test_weights = pd.read_csv('ddpg_test_weights.csv')
    td3_test_weights = pd.read_csv('td3_test_weights.csv')
    sac_test_weights = pd.read_csv('sac_test_weights.csv')
    df_close_full_stocks = pd.read_csv('datasets/close_prices.csv')
    %store -r filtered_stocks
    # %store -r df_close_full_stocks
    a2c_test_weights_dropdate = a2c_test_weights.drop(columns=['date'])
    ppo_test_weights_dropdate = ppo_test_weights.drop(columns=['date'])
    ddpg_test_weights_dropdate = ddpg_test_weights.drop(columns=['date'])
    td3_test_weights_dropdate = td3_test_weights.drop(columns=['date'])
    sac_test_weights_dropdate = sac_test_weights.drop(columns=['date'])
    all_test_weights = a2c_test_weights_dropdate + ppo_test_weights_dropdate + \
        ddpg_test_weights_dropdate + td3_test_weights_dropdate + sac_test_weights_dropdate
    proposed_method1_agents_test_weights_avg = all_test_weights / agents_count
    proposed_method1_agents_test_weights_avg.to_csv('proposed_method1_agents_test_weights_avg.csv', index=False)
    %store proposed_method1_agents_test_weights_avg

    start_date = '2021-03-29'
    end_date = '2024-03-28'
    filtered_df = df_close_full_stocks[(df_close_full_stocks['date'] < end_date) & (df_close_full_stocks['date'] >= start_date)]
    filtered_df.reset_index(drop=True, inplace=True)
    columns_to_drop = [col for col in filtered_df if col not in filtered_stocks]
    df_kept = filtered_df.drop(columns=columns_to_drop)
    df1 = df_kept.reindex(sorted(df_kept.columns), axis=1)
    test_close_normal = df1.pct_change()
    proposed_method1_eachstock_return = test_close_normal * proposed_method1_agents_test_weights_avg
    proposed_method1_eachstock_return.to_csv('proposed_method1_eachstock_return.csv', index=False)

    # Initialize a list to store row sums
    row_sums = []

    # Calculate row sums and store them
    for index, row in proposed_method1_eachstock_return.iterrows():
        row_sum = row.sum()
        row_sums.append(row_sum)

    # Create a new DataFrame with row sums
    proposed_method1_test_daily_return = pd.DataFrame({'daily_return': row_sums})
    proposed_method1_test_daily_return.insert(1, 'date', a2c_test_weights['date'])
    proposed_method1_test_daily_return = proposed_method1_test_daily_return[['date','daily_return']]
    proposed_method1_test_daily_return.to_csv('proposed_method1_test_daily_return.csv', index=False)
    %store proposed_method1_test_daily_return

    
    def load_and_merge_daily_returns(filenames, output_filename):
        dfs = []
        
        # Load each CSV file, drop the 'date' column, and store the DataFrame in a list
        for col_name, filename in filenames.items():
            df = pd.read_csv(filename)
            df = df.drop(columns=['date'])
            df.columns = [col_name]
            dfs.append(df)
        
        # Concatenate all DataFrames along the columns
        merged_df = pd.concat(dfs, axis=1)
        
        # Save the merged DataFrame to a CSV file
        merged_df.to_csv(output_filename, index=False)
        
        return merged_df

    # Dictionary of filenames and corresponding column names
    filenames = {
        'a2c': 'a2c_train_daily_return.csv',
        'ppo': 'ppo_train_daily_return.csv',
        'ddpg': 'ddpg_train_daily_return.csv',
        'td3': 'td3_train_daily_return.csv',
        'sac': 'sac_train_daily_return.csv'
    }

    # Output filename
    output_filename = 'merged_train_daily_return.csv'

    # Call the function and get the merged DataFrame
    merged_train_daily_return = load_and_merge_daily_returns(filenames, output_filename)

    merged_train_daily_return.to_csv('merged_train_daily_return.csv')

    merged_train_daily_return.plot(figsize=(10, 6))
    plt.title('Line Plot')
    plt.xlabel('X Axis Label')
    plt.ylabel('Y Axis Label')
    plt.grid(True)
    plt.show()

    def load_and_merge_test_returns(file_dict, output_filename):
        dfs = []
        
        # Load each CSV file, drop the 'Unnamed: 0' and 'date' columns, and store the DataFrame in a list
        for col_name, filename in file_dict.items():
            df = pd.read_csv(filename)
            if 'Unnamed: 0' in df.columns:
                df = df.drop(columns=['Unnamed: 0'])
            if 'date' in df.columns:
                df = df.drop(columns=['date'])
            df.columns = [col_name]
            dfs.append(df)
        
        # Concatenate all DataFrames along the columns
        merged_df = pd.concat(dfs, axis=1)
        
        # Save the merged DataFrame to a CSV file
        merged_df.to_csv(output_filename, index=False)
        
        return merged_df

    # Dictionary of filenames and corresponding column names
    file_dict = {
        'a2c': 'a2c_test_daily_return.csv',
        'ppo': 'ppo_test_daily_return.csv',
        'ddpg': 'ddpg_test_daily_return.csv',
        'td3': 'td3_test_daily_return.csv',
        'sac': 'sac_test_daily_return.csv'
    }

    # Output filename
    output_filename = 'merged_test_daily_return.csv'

    # Call the function and get the merged DataFrame
    merged_test_daily_return = load_and_merge_test_returns(file_dict, output_filename)

    merged_test_daily_return.to_csv('merged_test_daily_return.csv')

    merged_test_daily_return.plot(figsize=(10, 6))
    plt.title('Line Plot')
    plt.xlabel('X Axis Label')
    plt.ylabel('Y Axis Label')
    plt.grid(True)
    plt.show()

    columns_to_sum = ['a2c', 'ppo', 'ddpg', 'td3', 'sac']

    # Calculate ratios for each column in columns_to_sum
    for column in columns_to_sum:
        for i in range(0, len(merged_train_daily_return) - days + 1):
            sum_rows_allcolumns = merged_train_daily_return[columns_to_sum].iloc[i:i + days].sum().sum()
            sum_rows = merged_train_daily_return[column].iloc[i:i + days].sum()
            if sum_rows_allcolumns < 0.0005:
                sum_rows_allcolumns*=10
                ratio = sum_rows 
            else:
                ratio = sum_rows
            if i + days < len(merged_train_daily_return):
                merged_train_daily_return.loc[i + days, f'sumavg_{column}'] = ratio

    # Save the updated DataFrame to a CSV file
    merged_train_daily_return.to_csv('train_sum_avg.csv', index=False)
    merged_train_daily_return

    columns_to_sum = ['a2c', 'ppo', 'ddpg', 'td3', 'sac']

    # Calculate ratios for each column in columns_to_sum
    for column in columns_to_sum:
        for i in range(0, len(merged_test_daily_return)):
            sum_rows_allcolumns = merged_test_daily_return[columns_to_sum].iloc[i:i+days].sum().sum()
            sum_rows = merged_test_daily_return[column].iloc[i:i+days].sum()
            if sum_rows_allcolumns < 0.0005:
                sum_rows_allcolumns*=10
                ratio = sum_rows 
            else:
                ratio = sum_rows 
            #ratio = sum_rows / sum_rows_allcolumns if sum_rows_allcolumns != 0 else 0
            if i + days < len(merged_test_daily_return):
                merged_test_daily_return.loc[i + days, f'sumavg_{column}'] = ratio

    # Save the updated DataFrame to a CSV file
    merged_test_daily_return.to_csv('test_sum_avg.csv', index=False)
    merged_test_daily_return

    merged_train_daily_return_x = merged_train_daily_return.iloc[:, 0:5]
    merged_test_daily_return_x = merged_test_daily_return.iloc[:, 0:5]

    merged_train_daily_return_x.to_csv('merged_train_daily_return_x.csv')
    merged_train_daily_return_x

    merged_test_daily_return_x.to_csv('merged_test_daily_return_x.csv')
    merged_test_daily_return_x
    cumulative_rows = []
    for i in range(0, len(merged_train_daily_return_x)-days, 1):
        group = merged_train_daily_return_x.iloc[i:i+days]
        cumulative_row = group.iloc[0].tolist()
        for j in range(1, len(group)):
            cumulative_row.extend(group.iloc[j].tolist()[0:])
        cumulative_rows.append(cumulative_row)
    # Create a new DataFrame from the cumulative rows list
    my_train_X = pd.DataFrame(cumulative_rows)
    my_train_X.reset_index(drop=True, inplace=True)
    my_train_X.to_csv('my_train_X.csv')
    my_train_X

    cumulative_rows = []
    for i in range(0, len(merged_test_daily_return_x)-days, 1):
        group = merged_test_daily_return_x.iloc[i:i+days]
        cumulative_row = group.iloc[0].tolist()
        for j in range(1, len(group)):
            cumulative_row.extend(group.iloc[j].tolist()[0:])
        cumulative_rows.append(cumulative_row)
    # Create a new DataFrame from the cumulative rows list
    my_test_X = pd.DataFrame(cumulative_rows)
    my_test_X.reset_index(drop=True, inplace=True)
    my_test_X.to_csv('my_test_X.csv')
    my_test_X

    merged_train_daily_return_y = merged_train_daily_return.iloc[days:, 5:10]

    merged_train_daily_return_y.reset_index(drop=True, inplace=True)
    my_train_Y = merged_train_daily_return_y
    my_train_Y.to_csv('my_train_Y.csv')
    my_train_Y
    def assign_max(row):
        max_value = row.max()  # Find the maximum value in the row
        return [1 if val == max_value else 0 for val in row]

    # Apply the function to each row of the DataFrame and expand the result into separate columns
    max_df = my_train_Y.apply(assign_max, axis=1, result_type='expand')
    max_df.columns = ['a2c', 'ppo', 'ddpg', 'td3', 'sac']

    # max_df.to_csv('all_agents_predict_rank.csv')
    max_df.to_csv('my_train_Y_zero_one.csv')
    my_train_Y = max_df
    my_train_Y
    merged_test_daily_return_y = merged_test_daily_return.iloc[days:, 5:10]

    merged_test_daily_return_y.reset_index(drop=True, inplace=True)
    my_test_Y = merged_test_daily_return_y
    my_test_Y.to_csv('my_test_Y.csv')
    my_test_Y

    def assign_max(row):
        max_value = row.max()  # Find the maximum value in the row
        return [1 if val == max_value else 0 for val in row]
    max_df = my_test_Y.apply(assign_max, axis=1, result_type='expand')
    max_df.columns = ['a2c', 'ppo', 'ddpg', 'td3', 'sac']

    max_df.to_csv('my_test_Y_zero_one.csv')
    my_test_Y=max_df
    my_test_Y
    print('train_X shape: ',my_train_X.shape)
    print('train_Y shape: ',my_train_Y.shape)
    print('test_X shape: ',my_test_X.shape)
    print('test_Y shape: ',my_test_Y.shape)


    # Load the dataset files
    train_X = my_train_X
    test_X = my_test_X

    robust_scaler = RobustScaler()
    train_X_scaled = robust_scaler.fit_transform(train_X)
    test_X_scaled = robust_scaler.transform(test_X)


    # Convert back to DataFrame and save
    train_X_scaled_df = pd.DataFrame(train_X_scaled, columns=train_X.columns)
    test_X_scaled_df = pd.DataFrame(test_X_scaled, columns=test_X.columns)

    my_train_X = train_X_scaled_df
    my_test_X = test_X_scaled_df

    if neural_network == 'lstm':
        
        # Convert DataFrame columns to NumPy arrays
        my_train_X_array = my_train_X.to_numpy()
        my_test_X_array = my_test_X.to_numpy()
        my_train_Y_array = my_train_Y.to_numpy()
        my_test_Y_array = my_test_Y.to_numpy()

        # Split training data into training and validation sets
        train_X, val_X, train_Y, val_Y = train_test_split(my_train_X_array, my_train_Y_array, test_size=0.2, random_state=42)

        # Reshape the input data for LSTM
        train_X_reshaped = train_X.reshape((train_X.shape[0], 50, 1))
        val_X_reshaped = val_X.reshape((val_X.shape[0], 50, 1))
        test_X_reshaped = my_test_X_array.reshape((my_test_X_array.shape[0], 50, 1))

        # Define the LSTM model
        model = Sequential()
        model.add(LSTM(200, input_shape=(50, 1)))  # LSTM layer with 200 units
        model.add(Dense(1500, activation='relu'))  # Dense layer with 1500 units and ReLU activation
        model.add(Dense(5, activation='softmax'))  # Output layer with 5 units for 5 classes and softmax activation

        # Compile the model
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        model.summary()  # Print model summary

        # Define ModelCheckpoint callback to save the best model based on validation accuracy
        checkpoint = ModelCheckpoint('best_model_lstm.keras', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

        # Train the model with validation data
        history = model.fit(train_X_reshaped, train_Y, epochs=50, batch_size=256, validation_data=(val_X_reshaped, val_Y), callbacks=[checkpoint])

        # Evaluate the model on test data
        test_loss, test_accuracy = model.evaluate(test_X_reshaped, my_test_Y_array)
        print(f'Test Loss: {test_loss}')
        print(f'Test Accuracy: {test_accuracy}')

        # Predictions and saving to CSV
        predictions = model.predict(test_X_reshaped)
        final_prediction_shahin = pd.DataFrame(predictions)
        final_prediction_shahin.to_csv('main_proposed_method_prediction.csv')

        predicted_classes = np.argmax(predictions, axis=1)
        df_predicted = pd.DataFrame({'Predicted_Class': predicted_classes})
        df_predicted.to_csv('main_proposed_method_predicted_classes.csv', index=False)

        # One-hot encoded predictions
        num_classes = 5
        one_hot_predictions = np.eye(num_classes)[predicted_classes]
        np.savetxt('main_proposed_method_one_hot_predictions.csv', one_hot_predictions, delimiter=',')

        # Plotting training history
        plt.figure(figsize=(25, 10))

        # Accuracy plot
        plt.subplot(1, 2, 1)
        plt.plot(history.history['accuracy'], label='Train Accuracy')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.title('Model Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()

        # Loss plot
        plt.subplot(1, 2, 2)
        plt.plot(history.history['loss'], label='Train Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title('Model Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()

        # Save the figure before showing
        plt.savefig('./results/training_history.pdf')

        # Print predicted classes (optional)
        predicted_classes

    if neural_network == 'mlp':

        # Split data into training and validation sets
        train_X, val_X, train_Y, val_Y = train_test_split(my_train_X, my_train_Y, test_size=0.2, random_state=42)
    
        # Define the MLP model
        model = Sequential()
        model.add(Dense(2000, input_dim=50, activation='relu'))
        model.add(Dropout(0.4))
        model.add(Dense(5, activation='softmax'))
        # Compile the model
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        model.summary()

        # Define ModelCheckpoint callback to save the best model
        checkpoint = ModelCheckpoint('best_model_mlp.keras', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

        # Train the model with validation data
        history = model.fit(train_X, train_Y, epochs=2000, batch_size=2048, validation_data=(val_X, val_Y), callbacks=[checkpoint])

        # Load the best model saved by ModelCheckpoint
        best_model = load_model('best_model_mlp.keras')

        # Evaluate the best model on test data
        test_loss, test_accuracy = best_model.evaluate(my_test_X, my_test_Y)
        print(f'Test Loss: {test_loss}')
        print(f'Test Accuracy: {test_accuracy}')

        # Predictions and saving to CSV
        predictions = best_model.predict(my_test_X)
        final_prediction_shahin = pd.DataFrame(predictions)
        final_prediction_shahin.to_csv('main_proposed_method_prediction.csv')

        predicted_classes = np.argmax(predictions, axis=1)
        df_predicted = pd.DataFrame({'Predicted_Class': predicted_classes})
        df_predicted.to_csv('main_proposed_method_predicted_classes.csv', index=False)

        num_classes = 5
        one_hot_predictions = np.eye(num_classes)[predicted_classes]
        np.savetxt('main_proposed_method_one_hot_predictions.csv', one_hot_predictions, delimiter=',')

        predicted_classes


    # Assuming my_train_X, my_train_Y, my_test_X, and my_test_Y are already defined and in DataFrame format
    if neural_network == 'cnn':

        # Convert DataFrame to NumPy array and reshape data to add a channel dimension
        my_train_X = my_train_X.to_numpy().reshape(-1, 50, 1)
        my_test_X = my_test_X.to_numpy().reshape(-1, 50, 1)

        # Convert labels to NumPy array if they are in DataFrame format
        my_train_Y = my_train_Y.to_numpy()
        my_test_Y = my_test_Y.to_numpy()

        # Split data into training and validation sets
        train_X, val_X, train_Y, val_Y = train_test_split(my_train_X, my_train_Y, test_size=0.2, random_state=42)
    
        # Define the CNN model
        model = Sequential()
        model.add(Conv1D(64, 2, activation='relu', input_shape=(50, 1)))
        model.add(MaxPooling1D(2))
        model.add(Flatten())
        model.add(Dense(1024, activation='relu'))
        model.add(Dropout(0.8))
        model.add(Dense(5, activation='softmax'))

        # Compile the model
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        model.summary()

        # Define ModelCheckpoint callback to save the best model
        checkpoint = ModelCheckpoint('best_model_cnn.keras', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

        # Train the model with validation data
        history = model.fit(train_X, train_Y, epochs=100, batch_size=1024, validation_data=(val_X, val_Y), callbacks=[checkpoint])

        # Load the best model saved by ModelCheckpoint
        best_model = load_model('best_model_cnn.keras')

        # Evaluate the best model on test data
        test_loss, test_accuracy = best_model.evaluate(my_test_X, my_test_Y)
        print(f'Test Loss: {test_loss}')
        print(f'Test Accuracy: {test_accuracy}')

        # Predictions and saving to CSV
        predictions = best_model.predict(my_test_X)
        final_prediction_shahin = pd.DataFrame(predictions)
        final_prediction_shahin.to_csv('main_proposed_method_prediction.csv')

        predicted_classes = np.argmax(predictions, axis=1)
        df_predicted = pd.DataFrame({'Predicted_Class': predicted_classes})
        df_predicted.to_csv('main_proposed_method_predicted_classes.csv', index=False)

        num_classes = 5
        one_hot_predictions = np.eye(num_classes)[predicted_classes]
        np.savetxt('main_proposed_method_one_hot_predictions.csv', one_hot_predictions, delimiter=',')

        # Plotting training history
        plt.figure(figsize=(25, 10))

        # Accuracy plot
        plt.subplot(1, 2, 1)
        plt.plot(history.history['accuracy'], label='Train Accuracy')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.title('Model Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()

        # Loss plot
        plt.subplot(1, 2, 2)
        plt.plot(history.history['loss'], label='Train Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title('Model Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()

        # Save the figure before showing
        plt.savefig('./results/training_history.pdf')

    # Ensure the results directory exists
    os.makedirs('./results/model_mlp_result/', exist_ok=True)

    # Assuming my_train_X, my_train_Y, my_test_X, and my_test_Y are already defined and in DataFrame format
    if neural_network == 'dense_bn':

        # Convert DataFrame to NumPy array
        my_train_X = my_train_X.to_numpy()
        my_test_X = my_test_X.to_numpy()

        # Convert labels to NumPy array if they are in DataFrame format
        my_train_Y = my_train_Y.to_numpy()
        my_test_Y = my_test_Y.to_numpy()

        # Split data into training and validation sets
        train_X, val_X, train_Y, val_Y = train_test_split(my_train_X, my_train_Y, test_size=0.2, random_state=42)
    
        # Define the Dense model with Batch Normalization
        model = Sequential()
        model.add(Dense(256, input_dim=my_train_X.shape[1], activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.7))
        model.add(Dense(64, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.7))
        model.add(Dense(5, activation='softmax'))

        # Compile the model
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        model.summary()

        # Define ModelCheckpoint callback to save the best model
        checkpoint = ModelCheckpoint('best_model_dense_bn.keras', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
        
        # Train the model with validation data
        history = model.fit(train_X, train_Y, epochs=1500, batch_size=1024, validation_data=(val_X, val_Y), callbacks=[checkpoint])

        # Load the best model saved by ModelCheckpoint
        best_model = load_model('best_model_dense_bn.keras')

        # Evaluate the best model on test data
        test_loss, test_accuracy = best_model.evaluate(my_test_X, my_test_Y)
        print(f'Test Loss: {test_loss}')
        print(f'Test Accuracy: {test_accuracy}')

        # Predictions and saving to CSV
        predictions = best_model.predict(my_test_X)
        final_prediction_shahin = pd.DataFrame(predictions)
        final_prediction_shahin.to_csv('main_proposed_method_prediction.csv', index=False)

        predicted_classes = np.argmax(predictions, axis=1)
        df_predicted = pd.DataFrame({'Predicted_Class': predicted_classes})
        df_predicted.to_csv('main_proposed_method_predicted_classes.csv', index=False)

        num_classes = 5
        one_hot_predictions = np.eye(num_classes)[predicted_classes]
        np.savetxt('main_proposed_method_one_hot_predictions.csv', one_hot_predictions, delimiter=',')

        # Plotting training history
        plt.figure(figsize=(25, 10))

        # Accuracy plot
        plt.subplot(1, 2, 1)
        plt.plot(history.history['accuracy'], label='Train Accuracy')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.title('Model Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()

        # Loss plot
        plt.subplot(1, 2, 2)
        plt.plot(history.history['loss'], label='Train Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title('Model Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()

        # Save the figure before showing
        plt.savefig('./results/training_history.pdf')

    # Simple CNN model for the CIFAR-10 Dataset with improvements

    # Ensure the results directory exists
    os.makedirs('./results/model_cnn_result/', exist_ok=True)

    # Assuming my_train_X, my_train_Y, my_test_X, and my_test_Y are already defined and in DataFrame format
    if neural_network == 'conv2d':
        # Convert DataFrame to NumPy array
        my_train_X = my_train_X.values
        reshaped_array = my_train_X.reshape((3015, 5, 10, 1))
        my_train_X = reshaped_array

        my_test_X = my_test_X.values
        reshaped_array = my_test_X.reshape((745, 5, 10, 1))
        my_test_X = reshaped_array

        # Convert labels to NumPy array if they are in DataFrame format
        my_train_Y = my_train_Y.to_numpy()
        my_test_Y = my_test_Y.to_numpy()

        # Split data into training and validation sets
        train_X, val_X, train_Y, val_Y = train_test_split(my_train_X, my_train_Y, test_size=0.2, random_state=42)

        # Fix random seed for reproducibility
        seed = 7
        np.random.seed(seed)

        model = Sequential()
        model.add(Conv2D(256, (5, 5), input_shape=(5, 10, 1), activation='relu', padding='same'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(1, 1)))
        model.add(Dropout(0.5))
        model.add(Flatten())
        model.add(Dense(512, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))

        model.add(Dense(5, activation='softmax'))

        # Compile model
        epoch = 1000
        lrate = 0.001
        decay = lrate / epoch
        sgd = SGD(learning_rate=lrate, momentum=0.9, decay=decay, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

        print(model.summary())

        # Define ModelCheckpoint callback to save the best model
        checkpoint = ModelCheckpoint('best_model_cnn.keras', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
        
        # Train the model with validation data
        history = model.fit(train_X, train_Y, epochs=epoch, batch_size=1024, validation_data=(val_X, val_Y), callbacks=[checkpoint])

        # Load the best model saved by ModelCheckpoint
        best_model = load_model('best_model_cnn.keras')

        # Evaluate the best model on test data
        test_loss, test_accuracy = best_model.evaluate(my_test_X, my_test_Y)
        print(f'Test Loss: {test_loss}')
        print(f'Test Accuracy: {test_accuracy}')

        # Predictions and saving to CSV
        predictions = best_model.predict(my_test_X)
        final_prediction_shahin = pd.DataFrame(predictions)
        final_prediction_shahin.to_csv('final_prediction_shahin.csv', index=False)

        predicted_classes = np.argmax(predictions, axis=1)
        df_predicted = pd.DataFrame({'Predicted_Class': predicted_classes})
        df_predicted.to_csv('final_predicted_classes.csv', index=False)

        num_classes = 5
        one_hot_predictions = np.eye(num_classes)[predicted_classes]
        np.savetxt('final_one_hot_predictions.csv', one_hot_predictions, delimiter=',')

        # Plotting training history
        plt.figure(figsize=(25, 10))

        # Accuracy plot
        plt.subplot(1, 2, 1)
        plt.plot(history.history['accuracy'], label='Train Accuracy')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.title('Model Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()

        # Loss plot
        plt.subplot(1, 2, 2)
        plt.plot(history.history['loss'], label='Train Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title('Model Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()

        # Save the figure before showing
        plt.savefig('./results/training_history.pdf')

    if neural_network == 'dense_bn':
        my_dic_name = 'model_mlp_result'
    if neural_network == 'cnn':
        my_dic_name = 'model_cnn_result'
    if neural_network == 'lstm':
        my_dic_name = 'model_lstm_result'
    # Assuming my_train_X, my_train_Y, my_test_X, and my_test_Y are already defined and in DataFrame format
    # Assuming original column names are stored in a list original_columns

    # Ensure the results directory exists
    os.makedirs(f'./results/{my_dic_name}/', exist_ok=True)

    # Load the best model saved by ModelCheckpoint
    best_model = load_model('best_model_dense_bn.keras')

    # Predictions
    predictions = best_model.predict(my_test_X)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(my_test_Y, axis=1)

    # Evaluate the model on test data
    test_loss, test_accuracy = best_model.evaluate(my_test_X, my_test_Y)
    print(f'Test Loss: {test_loss}')
    print(f'Test Accuracy: {test_accuracy}')

    # Confusion Matrix
    conf_matrix = confusion_matrix(true_classes, predicted_classes)
    print("Confusion Matrix:")
    print(conf_matrix)

    # Plotting Confusion Matrix
    plt.figure(figsize=(10, 7))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.savefig(f'./results/{my_dic_name}/confusion_matrix.pdf')

    # Classification Report with new class names
    class_names = ['A2C', 'PPO', 'DDPG', 'TD3', 'SAC']
    class_report = classification_report(true_classes, predicted_classes, target_names=class_names)
    print("Classification Report:")
    print(class_report)

    # Check for class imbalance
    train_class_counts = np.sum(my_train_Y, axis=0)
    test_class_counts = np.sum(my_test_Y, axis=0)
    print(f"Train class distribution: {train_class_counts}")
    print(f"Test class distribution: {test_class_counts}")

    # Generate original columns based on the shape of my_train_X
    original_columns = [f'feature_{i}' for i in range(my_train_X.shape[1])]

    # Create DataFrames
    train_X_df = pd.DataFrame(my_train_X, columns=original_columns)
    test_X_df = pd.DataFrame(my_test_X, columns=original_columns)

    # Check for missing values
    print("Checking for missing values in train data...")
    print(train_X_df.isnull().sum())
    print("Checking for missing values in test data...")
    print(test_X_df.isnull().sum())

    # Investigate features with high variance
    print("Feature variance:")
    print(train_X_df.var())

    # Remove features with very high variance if needed
    high_variance_features = train_X_df.columns[train_X_df.var() > 1.5]  # Example threshold
    my_train_X_reduced = train_X_df.drop(columns=high_variance_features).to_numpy()
    my_test_X_reduced = test_X_df.drop(columns=high_variance_features).to_numpy()

    print("Reduced feature set:")
    print(my_train_X_reduced.shape)

    # Saving data to CSV files
    train_X_df.to_csv(f'./results/{my_dic_name}/train_X.csv', index=False)
    test_X_df.to_csv(f'./results/{my_dic_name}/test_X.csv', index=False)
    pd.DataFrame(my_train_Y, columns=class_names).to_csv(f'./results/{my_dic_name}/train_Y.csv', index=False)
    pd.DataFrame(my_test_Y, columns=class_names).to_csv(f'./results/{my_dic_name}/test_Y.csv', index=False)

    # ROC Curve plotting and saving data
    n_classes = my_test_Y.shape[1]
    roc_data = pd.DataFrame()
    auc_values = []

    # Compute ROC curve and ROC area for each class
    for i in range(n_classes):
        fpr, tpr, _ = roc_curve(my_test_Y[:, i], predictions[:, i])
        roc_auc = auc(fpr, tpr)
        auc_values.append(roc_auc)
        
        class_roc_data = pd.DataFrame({
            f'False Positive Rate Class {i}': fpr,
            f'True Positive Rate Class {i}': tpr
        })
        
        # Merge class ROC data with main ROC data
        roc_data = pd.concat([roc_data, class_roc_data], axis=1)

    # Save ROC curve data to CSV
    roc_data.to_csv(f'./results/{my_dic_name}/roc_curve_data.csv', index=False)

    # Save AUC values to CSV
    auc_df = pd.DataFrame({'Class': class_names, 'AUC': auc_values})
    auc_df.to_csv(f'./results/{my_dic_name}/auc_values.csv', index=False)

    # Plotting ROC curve for each class
    plt.figure(figsize=(10, 7))
    for i in range(n_classes):
        plt.plot(roc_data[f'False Positive Rate Class {i}'], roc_data[f'True Positive Rate Class {i}'], lw=2, label=f'Class {class_names[i]} (area = {auc_values[i]:.2f})')

    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc='lower right')
    plt.savefig(f'./results/{my_dic_name}/roc.pdf')

    main_proposed_method_prediction = pd.read_csv('main_proposed_method_prediction.csv', header=0)
    main_proposed_method_prediction = main_proposed_method_prediction.rename(columns={'0': 'a2c','1':'ppo','2':'ddpg','3':'td3','4':'sac','5':'loss_day'})
    zeros_df = pd.DataFrame(np.zeros((days, main_proposed_method_prediction.shape[1])), columns=main_proposed_method_prediction.columns)
    zeros_df['td3']=0
    main_proposed_method_prediction = pd.concat([zeros_df, main_proposed_method_prediction], ignore_index=True)

    ranked_df = main_proposed_method_prediction

    def assign_max(row):
        max_value = row.max()  # Find the maximum value in the row
        if max_value == 0:
            return [0] + [0] * (len(row) - 1)
        else:
            # Assign 1 to the max value and 0 to others
            return [1 if val == max_value else 0 for val in row]


    # Apply the function to each row of the DataFrame and expand the result into separate columns
    max_df = ranked_df.apply(assign_max, axis=1, result_type='expand')
    max_df.columns = ['a2c', 'ppo', 'ddpg', 'td3', 'sac']

    max_df.to_csv('qwerty.csv')
    max_df
    a2c_test_weights2 = pd.read_csv('a2c_test_weights.csv')
    ppo_test_weights2 = pd.read_csv('ppo_test_weights.csv')
    ddpg_test_weights2 = pd.read_csv('ddpg_test_weights.csv')
    td3_test_weights2 = pd.read_csv('td3_test_weights.csv')
    sac_test_weights2 = pd.read_csv('sac_test_weights.csv')

    a2c_test_weights = a2c_test_weights2.drop(columns=['date'])
    ppo_test_weights = ppo_test_weights2.drop(columns=['date'])
    ddpg_test_weights = ddpg_test_weights2.drop(columns=['date'])
    td3_test_weights = td3_test_weights2.drop(columns=['date'])
    sac_test_weights = sac_test_weights2.drop(columns=['date'])

    a2c_test_weights.reset_index(drop=True, inplace=True)
    ppo_test_weights.reset_index(drop=True, inplace=True)
    ddpg_test_weights.reset_index(drop=True, inplace=True)
    td3_test_weights.reset_index(drop=True, inplace=True)
    sac_test_weights.reset_index(drop=True, inplace=True)

    a2c_test_weights
    a2c_merged_df = pd.concat([a2c_test_weights, max_df.iloc[:, 0]], axis=1)
    ppo_merged_df = pd.concat([ppo_test_weights, max_df.iloc[:, 1]], axis=1)
    ddpg_merged_df = pd.concat([ddpg_test_weights, max_df.iloc[:, 2]], axis=1)
    td3_merged_df = pd.concat([td3_test_weights, max_df.iloc[:, 3]], axis=1)
    sac_merged_df = pd.concat([sac_test_weights, max_df.iloc[:, 4]], axis=1)

    td3_merged_df
    a2c_merged_df.iloc[:, 0:]
    a2c_merged_df.iloc[:, :-1] *= a2c_merged_df.iloc[:, -1].values[:, None]
    ppo_merged_df.iloc[:, :-1] *= ppo_merged_df.iloc[:, -1].values[:, None]
    ddpg_merged_df.iloc[:, :-1] *= ddpg_merged_df.iloc[:, -1].values[:, None]
    td3_merged_df.iloc[:, :-1] *= td3_merged_df.iloc[:, -1].values[:, None]
    sac_merged_df.iloc[:, :-1] *= sac_merged_df.iloc[:, -1].values[:, None]

    a2c_merged_df = a2c_merged_df.drop(columns=['a2c'])
    ppo_merged_df = ppo_merged_df.drop(columns=['ppo'])
    ddpg_merged_df = ddpg_merged_df.drop(columns=['ddpg'])
    td3_merged_df = td3_merged_df.drop(columns=['td3'])
    sac_merged_df = sac_merged_df.drop(columns=['sac'])


    a2c_merged_df
    dfs = [a2c_merged_df, ppo_merged_df, ddpg_merged_df, td3_merged_df, sac_merged_df]

    # Extract columns except the first column ("date")
    cols_to_sum = dfs[0].columns[0:]
    # Initialize an empty DataFrame to store the summed values
    above_sum_df6 = pd.DataFrame(columns=cols_to_sum)
    # Iterate over each DataFrame and sum corresponding columns
    for df in dfs:
        above_sum_df6 = above_sum_df6.add(df[cols_to_sum], fill_value=0)
    # Add the "date" column from df1 to the summed values DataFrame

    above_sum_df6.to_csv('./results/main_proposed_method/test_weights.csv')
    above_sum_df6
    percent_change_df = above_sum_df6.pct_change()
    percent_change_df.to_csv('./results/main_proposed_method/percent_change.csv')
    %store -r filtered_stocks
    start_date = '2021-03-29'
    end_date = '2024-03-28'
    filtered_df = df_close_full_stocks[(df_close_full_stocks['date'] < end_date) & (df_close_full_stocks['date'] >= start_date)]
    filtered_df.reset_index(drop=True, inplace=True)
    columns_to_drop = [col for col in filtered_df if col not in filtered_stocks]
    df_kept = filtered_df.drop(columns=columns_to_drop)
    df1 = df_kept.reindex(sorted(df_kept.columns), axis=1)
    test_close_normal = df1.pct_change()
    hoseini_algorithm = above_sum_df6
    hosini_daily_return = test_close_normal * hoseini_algorithm
    hosini_daily_return.to_csv('hosini_daily_return.csv', index=False)
    hosini_daily_return

    def load_csv_files(filenames):
        dataframes = {}
        for name, filename in filenames.items():
            df = pd.read_csv(filename)
            if 'Unnamed: 0' in df.columns:
                df = df.drop(columns=['Unnamed: 0'])
            dataframes[name] = df
        return dataframes

    # Dictionary of filenames
    filenames = {
        'a2c_train_daily_return': 'a2c_train_daily_return.csv',
        'ppo_train_daily_return': 'ppo_train_daily_return.csv',
        'ddpg_train_daily_return': 'ddpg_train_daily_return.csv',
        'td3_train_daily_return': 'td3_train_daily_return.csv',
        'sac_train_daily_return': 'sac_train_daily_return.csv',
        'a2c_test_returns': 'a2c_test_daily_return.csv',
        'ppo_test_returns': 'ppo_test_daily_return.csv',
        'ddpg_test_returns': 'ddpg_test_daily_return.csv',
        'td3_test_returns': 'td3_test_daily_return.csv',
        'sac_test_returns': 'sac_test_daily_return.csv',
        'proposed_method1_test_daily_return': 'proposed_method1_test_daily_return.csv',
    }

    # Load the data
    dataframes = load_csv_files(filenames)

    # Access individual DataFrames
    a2c_train_daily_return = dataframes['a2c_train_daily_return']
    ppo_train_daily_return = dataframes['ppo_train_daily_return']
    ddpg_train_daily_return = dataframes['ddpg_train_daily_return']
    td3_train_daily_return = dataframes['td3_train_daily_return']
    sac_train_daily_return = dataframes['sac_train_daily_return']
    a2c_test_returns = dataframes['a2c_test_returns']
    ppo_test_returns = dataframes['ppo_test_returns']
    ddpg_test_returns = dataframes['ddpg_test_returns']
    td3_test_returns = dataframes['td3_test_returns']
    sac_test_returns = dataframes['sac_test_returns']
    proposed_method1_test_daily_return = dataframes['proposed_method1_test_daily_return']

    %store a2c_train_daily_return
    %store ppo_train_daily_return
    %store ddpg_train_daily_return
    %store td3_train_daily_return
    %store sac_train_daily_return

    %store a2c_test_returns
    %store ppo_test_returns
    %store ddpg_test_returns
    %store td3_test_returns
    %store sac_test_returns

    %store proposed_method1_test_daily_return

    %store -r prices_train_df
    %store -r prices_test_df

    a2c_train_cum_returns = (
        1 + a2c_train_daily_return.reset_index(drop=True).set_index(['date'])).cumprod()
    a2c_train_cum_returns = a2c_train_cum_returns['daily_return']
    a2c_train_cum_returns.name = 'Portfolio 1: A2C Model'
    ppo_train_cum_returns = (
        1 + ppo_train_daily_return.reset_index(drop=True).set_index(['date'])).cumprod()
    ppo_train_cum_returns = ppo_train_cum_returns['daily_return']
    ppo_train_cum_returns.name = 'Portfolio 2: PPO Model'
    ddpg_train_cum_returns = (
        1 + ddpg_train_daily_return.reset_index(drop=True).set_index(['date'])).cumprod()
    ddpg_train_cum_returns = ddpg_train_cum_returns['daily_return']
    ddpg_train_cum_returns.name = 'Portfolio 3: DDPG Model'
    td3_train_cum_returns = (
        1 + td3_train_daily_return.reset_index(drop=True).set_index(['date'])).cumprod()
    td3_train_cum_returns = td3_train_cum_returns['daily_return']
    td3_train_cum_returns.name = 'Portfolio 4: TD3 Model'
    sac_train_cum_returns = (
        1 + sac_train_daily_return.reset_index(drop=True).set_index(['date'])).cumprod()
    sac_train_cum_returns = sac_train_cum_returns['daily_return']
    sac_train_cum_returns.name = 'Portfolio 5: SAC Model'
    date_list = list(ddpg_train_cum_returns.index)

    fig, ax = plt.subplots(figsize=(25, 10))

    a2c_train_cum_returns.plot(ax=ax, color='gray', alpha=0.4)
    ppo_train_cum_returns.plot(ax=ax, color='green', alpha=0.4)
    ddpg_train_cum_returns.plot(ax=ax, color='purple', alpha=0.4)
    td3_train_cum_returns.plot(ax=ax, color='red', alpha=0.4)
    sac_train_cum_returns.plot(ax=ax, color='blue', alpha=0.4)
    plt.legend(loc="best", fontsize=19, handlelength=4,
            handleheight=2, handletextpad=2)
    plt.grid(True)
    ax.set_ylabel("cummulative return", fontsize=30)
    ax.set_xlabel("Date", fontsize=28)
    ax.tick_params(axis='x', labelsize=25)
    ax.tick_params(axis='y', labelsize=25)
    ax.set_title(
        "Backtest based on the data from 2009-03-20 to 2021-03-26", fontsize=30)

    fig.savefig('results/result_train/back_test_on_train_data.pdf')
    a2c_train_cum_returns.to_csv('results/result_train/a2c_train_cum_return.csv')
    ppo_train_cum_returns.to_csv('results/result_train/ppo_train_cum_return.csv')
    ddpg_train_cum_returns.to_csv('results/result_train/ddpg_train_cum_return.csv')
    td3_train_cum_returns.to_csv('results/result_train/td3_train_cum_return.csv')
    sac_train_cum_returns.to_csv('results/result_train/sac_train_cum_return.csv')

    main_proposed_method_eachstock_return = pd.read_csv(
        'hosini_daily_return.csv')
    main_proposed_method_eachstock_return['daily_return'] = main_proposed_method_eachstock_return.sum(axis=1)

    a2c_test_cum_returns = (1 + a2c_test_returns['daily_return']).cumprod()
    a2c_test_cum_returns.name = 'Portfolio 1: A2C Model'

    ppo_test_cum_returns = (1 + ppo_test_returns['daily_return']).cumprod()
    ppo_test_cum_returns.name = 'Portfolio 2: PPO Model'

    ddpg_test_cum_returns = (1 + ddpg_test_returns['daily_return']).cumprod()
    ddpg_test_cum_returns.name = 'Portfolio 3: DDPG Model'

    td3_test_cum_returns = (1 + td3_test_returns['daily_return']).cumprod()
    td3_test_cum_returns.name = 'Portfolio 4: TD3 Model'

    sac_test_cum_returns = (1 + sac_test_returns['daily_return']).cumprod()
    sac_test_cum_returns.name = 'Portfolio 5: SAC Model'


    proposed_method1_test_cum_return = (
        1 + proposed_method1_test_daily_return['daily_return']).cumprod()
    proposed_method1_test_cum_return.name = 'Portfolio 6: Proposed method 1'

    main_proposed_method_cum_return = (
        1 + main_proposed_method_eachstock_return['daily_return']).cumprod()
    main_proposed_method_cum_return.name = 'Portfolio 7: Main proposed method'


    # Plot the culmulative returns of the portfolios
    fig, ax = plt.subplots(figsize=(25,10))
    a2c_test_cum_returns.plot(ax=ax, color='blue', alpha=.4);
    ppo_test_cum_returns.plot(ax=ax, color='green', alpha=.4);
    ddpg_test_cum_returns.plot(ax=ax, color='purple', alpha=.4);
    td3_test_cum_returns.plot(ax=ax, color='red', alpha=0.4);
    sac_test_cum_returns.plot(ax=ax, color='darkred', alpha=0.4);
    proposed_method1_test_cum_return.plot(ax=ax, color='darkred', alpha=0.4)
    main_proposed_method_cum_return.plot(ax=ax, color='black', alpha=0.8)
    plt.legend(loc="best", fontsize=17, handlelength=4, handleheight=1,handletextpad=2);
    plt.grid(True);
    ax.set_ylabel("cummulative return", fontsize = 30);
    ax.set_xlabel("Date", fontsize=28);
    ax.set_title("Backtest based on the data from 2021-03-29 to 2024-03-28", fontsize=30);
    ax.tick_params(axis='x', labelsize=25);
    ax.tick_params(axis='y', labelsize=25);

    fig.savefig('./results/result_test/back_test_on_test_data.pdf');
    a2c_test_cum_returns.to_csv('./results/result_test/a2c_test_cum_returns.csv')
    ppo_test_cum_returns.to_csv('./results/result_test/ppo_test_cum_returns.csv')
    ddpg_test_cum_returns.to_csv('./results/result_test/ddpg_test_cum_returns.csv')
    td3_test_cum_returns.to_csv('./results/result_test/td3_test_cum_returns.csv')
    sac_test_cum_returns.to_csv('./results/result_test/sac_test_cum_returns.csv')
    proposed_method1_test_cum_return.to_csv(
        './results/result_test/proposed_method1_test_cum_return.csv')
    main_proposed_method_cum_return.to_csv(
        './results/result_test/main_proposed_method_cum_return.csv')
    # Define a Function for Getting the Portfolio Statistics

    def portfolio_stats(portfolio_returns):
        # Pass the returns into a dataframe
        port_rets_df = pd.DataFrame(portfolio_returns)
        port_rets_df = port_rets_df.reset_index()
        port_rets_df.columns = ['date', 'daily_return']

        # Use the FinRL Library to get the Portfolio Returns
        # This makes use of the Pyfolio Library

        DRL_strat = backtest_strat(port_rets_df)
        perf_func = timeseries.perf_stats
        perf_stats_all = perf_func(returns=DRL_strat,
                                factor_returns=DRL_strat,
                                positions=None, transactions=None, turnover_denom="AGB")
        perf_stats_all = pd.DataFrame(perf_stats_all)
        perf_stats_all.columns = ['Statistic']
        return perf_stats_all
    # Get the Portfolio Statistics for all the portfolios
    portfolios_returns_dict = {'A2C Model': a2c_test_returns['daily_return'],
                            'PPO Model': ppo_test_returns['daily_return'],
                            'DDPG Model': ddpg_test_returns['daily_return'],
                            'TD3 Model': td3_test_returns['daily_return'],
                            'SAC Model': sac_test_returns['daily_return'],
                            'Proposed method1': proposed_method1_test_daily_return['daily_return'],
                            'Main proposed method': main_proposed_method_eachstock_return['daily_return']
                            }

    portfolios_stats = pd.DataFrame()
    for i, j in portfolios_returns_dict.items():
        port_stats = portfolio_stats(j)
        portfolios_stats[i] = port_stats['Statistic']
    portfolios_stats.to_csv('./results/result_portfolios/portfolios_stats.csv')
    portfolios_stats
    a2c_test_cum_returns = pd.read_csv('./results/result_test/a2c_test_cum_returns.csv')
    a2c_test_cum_returns = a2c_test_cum_returns.drop(columns=['Unnamed: 0'])
    last_column1 = a2c_test_cum_returns.iloc[-1]
    ppo_test_cum_returns = pd.read_csv('./results/result_test/ppo_test_cum_returns.csv')
    ppo_test_cum_returns = ppo_test_cum_returns.drop(columns=['Unnamed: 0'])
    last_column2 = ppo_test_cum_returns.iloc[-1]
    ddpg_test_cum_returns = pd.read_csv('./results/result_test/ddpg_test_cum_returns.csv')
    ddpg_test_cum_returns = ddpg_test_cum_returns.drop(columns=['Unnamed: 0'])
    last_column3 = ddpg_test_cum_returns.iloc[-1]
    td3_test_cum_returns = pd.read_csv('./results/result_test/td3_test_cum_returns.csv')
    td3_test_cum_returns = td3_test_cum_returns.drop(columns=['Unnamed: 0'])
    last_column4 = td3_test_cum_returns.iloc[-1]
    sac_test_cum_returns = pd.read_csv('./results/result_test/sac_test_cum_returns.csv')
    sac_test_cum_returns = sac_test_cum_returns.drop(columns=['Unnamed: 0'])
    last_column5 = sac_test_cum_returns.iloc[-1]
    main_proposed_method_cum_return = pd.read_csv('./results/result_test/main_proposed_method_cum_return.csv')
    main_proposed_method_cum_return = main_proposed_method_cum_return.drop(columns=['Unnamed: 0'])
    last_column6 = main_proposed_method_cum_return.iloc[-1]

    maximum = max(last_column1[0], last_column2[0],last_column3[0],last_column4[0],last_column5[0])

    if last_column6[0] > maximum:
        best_return=False

